In [1]:
#setup nice plotting
%matplotlib widget

# we import useful libraries
import time
import numpy as np
import pinocchio
import os
import meshcat
import matplotlib as mp
import matplotlib.pyplot as plt
import scipy


np.set_printoptions(suppress=True, precision=4) # we don't want to print every decimal!

In [5]:
# the directory where the robot models are located
package_dirs = './urdf/'

# the name of the URDF model (the robot model)
urdf_file = 'iiwa.urdf'
END_EFF_FRAME_ID = 17 # number of the frame corresponding to the end-effector

# we load the urdf models with Pinocchio (it will build a robot model, its kinematic tree, etc)
urdf = package_dirs + urdf_file
robot = pinocchio.RobotWrapper.BuildFromURDF(urdf, package_dirs)

# we create the visualization for Meshcat
viz = pinocchio.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
try:
    viz.initViewer(open=True)
except ImportError as err:
    print("Error while initializing the viewer. It seems you should install Python meshcat")
    print(err)
    sys.exit(0)
viz.loadViewerModel()

q = pinocchio.neutral(robot.model)
viz.display(q) 

# create a ball to visualize the goal
viz.viewer['ball'].set_object(meshcat.geometry.Sphere(0.065), 
                              meshcat.geometry.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [7]:
end_eff_desired = np.array([0.5,.6, 0.2])
viz.viewer['ball'].set_transform(meshcat.transformations.translation_matrix(end_eff_desired))

In [13]:
from_q = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1])
to_q = np.array([0.1,0.1,0.1,0.1,0.1,0.1,0.1])

# q = pinocchio.neutral(robot.model)
viz.display(q)
time.sleep(0.001)